In [1]:
# Importando pacotes
from tkinter import *
from tkinter import ttk
import numpy as np
import pandas as pd
from sklearn import preprocessing
from math import*


root = Tk()
root.geometry("1200x650")
root.title('Comparador de similaridades entre poços')


#BACKGROUND
fundo=PhotoImage(file='background/back19.png',master=root)
canv=Canvas(root,width=1200,height=650)
canv.pack(fill='both',expand=True)
canv.create_image(0,0,image=fundo,anchor='nw')


#CRIANDO OS FRAMES ONDE SERÃO INSERIDOS OS BOTÕES, CHECK BOX E RADIO BUTTONS
meu_frame1=LabelFrame(canv, text= 'Poço a ser comparado',padx=0,pady=0,fg='#1c4587')
meu_frame2=LabelFrame(canv, text= 'Variáveis a serem comparadas',padx=0,pady=0,fg='#1c4587')
meu_frame3=LabelFrame(canv, text= 'Método de similaridade',padx=0,pady=0,fg='#1c4587')
meu_frame4=LabelFrame(canv, text= 'Váriáveis de filtragem',padx=0,pady=0,fg='#1c4587')

canv.create_window(0,0,window=meu_frame1,anchor='nw')
canv.create_window(0,260,window=meu_frame2,anchor='w',width=570,height=180)
canv.create_window(0,450,window=meu_frame3,anchor='w')
canv.create_window(400,0,window=meu_frame4,anchor='n')


#INSERINDO QUADRO DE TEXTO PARA OUTPUT
tex = Text(master=canv,width=50,height=40,bd=3)
canv.create_window(1200,0,window=tex,anchor='ne')
tex.tag_config('warning', background="yellow", foreground="red")

#PREPARANDO O AMBIENTE DE ENTRADA
df=pd.read_csv('base_final.csv').set_index('NOME')
aux=['TIPO','NFASES','FORMATO']
aux2= np.isin(df.columns,aux)
aux2=aux2*(-1)+1
aux2=aux2.astype(bool)
numericos=df.columns[aux2]

df_final=df.select_dtypes(include=np.number)
    

#FUNÇÕES

def raiz(x): #CALCULA A RAIZ DA SOMA DO QUADRADO DE UM VETOR
    return sqrt(sum([a*a for a in x]))
 
def cosseno(x,y): # CALCULA A SIMILARIDADE POR COSSENO ENTRE DOIS POÇOS
    n1=np.array(x)
    n2=np.array(y)
    numerador = np.dot(n1,n2)
    denominador = raiz(x)*raiz(y)
    return round(numerador/denominador,6)
 

def leitura(): #RELÊ A BASE DE ENTRADA PARA REINICIAR A ANÁLISE
    global df,numericos,df_final
    df=pd.read_csv('base_final.csv').set_index('NOME')
    aux=['TIPO','NFASES','FORMATO']
    aux2= np.isin(df.columns,aux)
    aux2=aux2*(-1)+1
    aux2=aux2.astype(bool)
    numericos=df.columns[aux2]

    df_final=df.select_dtypes(include=np.number)

    
def filtra_igual(col,df,pit,std=1): #FILTRA OS POÇOS QUE POSSUEM 'NFASES', 'TIPO' OU 'FORMATO' IDÊNTICOS 
                                    #AO POÇO ANALISADO
    df2=df.loc[pit]
    for cols in col:
        if cols not in numericos:
            df=df[df[cols]==df2[cols]]
        else:
            aux1=df.loc[pit][cols]-(std*df[cols].std())
            aux2=df.loc[pit][cols]+(std*df[cols].std())
            df=df[(df[cols]>aux1).values * (df[cols]<aux2).values]
    return(df)



# FUNÇÃO PARA GERAR OS RESULTADOS
# A FUNÇÃO NORMALIZA OS DADOS NUMÉRICOS E DEPOIS CALCULA OU A DISTÂNCIA EUCLIDIANA OU A SIMILARIDADE POR COSSENO
# A DISTÂNCIA EUCLIDIANA CALCULA A PROXIMIDADE ENTRE DOIS PONTOS, ENQUANTO A SIMILARIDADE POR COSSENO CALCULA
# A SEMELHANÇA ENTRE A DIREÇÃO DE DOIS VETORES, OU SEJA, SE DOIS VETORES SÃO SEMELHANTES POR PROPORÇÃO.

def resultado():
    
    #scaler = preprocessing.StandardScaler()
    #scaled_df=scaler.fit_transform(df[numericos])
    x_array =  np.array(df.reset_index()[numericos])
    scaled_df = preprocessing.normalize(x_array)
    scaled_df = pd.DataFrame(scaled_df, columns=numericos)
    scaled_df['NOME']=df.index
    scaled_df.set_index('NOME',inplace=True)
    if len(fil)>0:
        scaled_df=scaled_df[fil]
        
    
    dist=[]
    if (rd==1):
        for i in scaled_df.index:
            dist.append(np.linalg.norm(scaled_df.loc[pit] - scaled_df.loc[[i]]))
        df_final2= scaled_df[scaled_df.columns].copy()
        df_final2['DISTANCIA']=dist
    
        exclui=df_final2.index==pit
        df_final2=df_final2[~exclui]
        tex.insert(END,"\n")
        tex.insert(END,"Poços mais similares pela Distância Euclidiana")
        tex.insert(END,"\n")
        tex.insert(END,df_final2[['DISTANCIA']].sort_values(by=['DISTANCIA']).head(4))
        tex.insert(END,"\n")
        tex.insert(END,"\n")
        tex.insert(END,'Distância Média: '+round(df_final2['DISTANCIA'].describe()[1],6).astype('str'))
        tex.insert(END,"\n")
        tex.insert(END,'Distância Máxima: '+round(df_final2['DISTANCIA'].describe()[7],6).astype('str'))
        tex.insert(END,"\n")
        if df_final2.shape[0]<4:
            tex.insert(END,"\n")
            tex.insert(END,"A filtragem aplicada gerou menos que 4 poços. Pressione 'Reiniciar' e aplique menos filtros",'warning')
            tex.insert(END,"\n")
    else:
        for i in scaled_df.index:
            dist.append(cosseno(list(scaled_df.loc[pit]),list(scaled_df.loc[i])))
        df_final2= scaled_df[scaled_df.columns].copy()
        df_final2['COSSENO']=dist
        
        exclui=df_final2.index==pit
        df_final2=df_final2[~exclui]
        tex.insert(END,"\n")
        tex.insert(END,"Poços mais similares pela Similaridade por Cosseno")
        tex.insert(END,"\n")
        tex.insert(END,df_final2[['COSSENO']].sort_values(by=['COSSENO'],ascending=False).head(4))
        tex.insert(END,"\n")
        tex.insert(END,"\n")
        tex.insert(END,'Similaridade Média: '+round(df_final2['COSSENO'].describe()[1],6).astype('str'))
        tex.insert(END,"\n")
        tex.insert(END,'Similaridade Mínima: '+round(df_final2['COSSENO'].describe()[3],6).astype('str'))
        tex.insert(END,"\n")
        if df_final2.shape[0]<4:
            tex.insert(END,"\n")
            tex.insert(END,"A filtragem aplicada gerou menos que 4 poços. Pressione 'Reiniciar' e aplique menos filtros",'warning')
            tex.insert(END,"\n")
            

def filtro(): #FILTRA AS VARIÁVEIS QUE SERÃO CONSIDERADAS NA COMPARAÇÃO
    global fil
    fil0=var0.get();fil1=var1.get();fil2=var2.get();fil3=var3.get();fil4=var4.get();fil5=var5.get()
    fil6=var6.get();fil7=var7.get();fil8=var8.get();fil9=var9.get();fil10=var10.get();fil11=var11.get()
    fil12=var12.get();fil13=var13.get();fil14=var14.get();fil15=var15.get();fil16=var16.get()
    fil17=var17.get();fil18=var18.get()
    
    fil=[fil0,fil1,fil2,fil3,fil4,fil5,fil6,fil7,fil8,fil9,
        fil10,fil11,fil12,fil13,fil14,fil15,fil16,fil17,fil18]
    fil=list(filter(lambda x: x!='0', fil))
    return fil
    


def filtro2(): # RETORNA AS VARIÁVEIS NÃO NUMÉRICAS QUE SERÃO FILTRADAS
    global filb,df
    filb19=var19.get()
    filb20=var20.get()
    filb21=var21.get()
    filb=[filb19,filb20,filb21]
    filb=list(filter(lambda x: x!='0', filb))
    if len(filb)>0:
        df=filtra_igual(filb,df,pit)
    return df


def pega(): # RETORNA O POÇO SELECIONADO
    global pit
    pit=Combo.get()
    

def texto(): # RETORNA OS OUTPUTS NA CAIXA DE TEXTO
    tex.insert(END,df_final.loc[pit])

rd=1
def radio(): # RETORNA O MÉTODO DE SIMILARIDADE ESCOLHIDO
    global rd
    rd=rb.get()
    
 

 #DECLARANDO VARIAVEIS
var0=Variable();var1=Variable();var2=Variable();var3=Variable();var4=Variable();var5=Variable();var6=Variable()
var7=Variable();var8=Variable();var9=Variable();var10=Variable();var11=Variable();var12=Variable()
var13=Variable();var14=Variable();var15=Variable();var16=Variable();var17=Variable();var18=Variable()
var19=Variable();var20=Variable();var21=Variable()



#INSERINDO CHECK BUTTONS
cols=list(df_final[numericos].columns)
c0=Checkbutton(meu_frame2, text=cols[0], variable= var0, onvalue=cols[0],width=16,anchor='w')
c0.select()
c1=Checkbutton(meu_frame2, text=cols[1], variable= var1, onvalue=cols[1],width=16,anchor='w')
c1.select()
c2=Checkbutton(meu_frame2, text=cols[2], variable= var2, onvalue=cols[2],width=16,anchor='w')
c2.select()
c3=Checkbutton(meu_frame2, text=cols[3], variable= var3, onvalue=cols[3],width=16,anchor='w')
c3.select()
c4=Checkbutton(meu_frame2, text=cols[4], variable= var4, onvalue=cols[4],width=16,anchor='w')
c4.select()
c5=Checkbutton(meu_frame2, text=cols[5], variable= var5, onvalue=cols[5],width=16,anchor='w')
c5.select()
c6=Checkbutton(meu_frame2, text=cols[6], variable= var6, onvalue=cols[6],width=16,anchor='w')
c6.select()
c7=Checkbutton(meu_frame2, text=cols[7], variable= var7, onvalue=cols[7],width=16,anchor='w')
c7.select()
c8=Checkbutton(meu_frame2, text=cols[8], variable= var8, onvalue=cols[8],width=16,anchor='w')
c8.select()
c9=Checkbutton(meu_frame2, text=cols[9], variable= var9, onvalue=cols[9],width=16,anchor='w')
c9.select()
c10=Checkbutton(meu_frame2, text=cols[10], variable= var10, onvalue=cols[10],width=16,anchor='w')
c10.select()
c11=Checkbutton(meu_frame2, text=cols[11], variable= var11, onvalue=cols[11],width=16,anchor='w')
c11.select()
c12=Checkbutton(meu_frame2, text=cols[12], variable= var12, onvalue=cols[12],width=16,anchor='w')
c12.select()
c13=Checkbutton(meu_frame2, text=cols[13], variable= var13, onvalue=cols[13],width=16,anchor='w')
c13.select()
c14=Checkbutton(meu_frame2, text=cols[14], variable= var14, onvalue=cols[14],width=16,anchor='w')
c14.select()
c15=Checkbutton(meu_frame2, text=cols[15], variable= var15, onvalue=cols[15],width=16,anchor='w')
c15.select()
c16=Checkbutton(meu_frame2, text=cols[16], variable= var16, onvalue=cols[16],width=16,anchor='w')
c16.select()
c17=Checkbutton(meu_frame2, text=cols[17], variable= var17, onvalue=cols[17],width=16,anchor='w')
c17.select()
c18=Checkbutton(meu_frame2, text=cols[18], variable= var18, onvalue=cols[18],width=16,anchor='w')
c18.select()
c19=Checkbutton(meu_frame4, text="Filtrar por Tipo", variable= var19, onvalue='TIPO',offvalue='0')
c19.deselect()
c20=Checkbutton(meu_frame4, text="Filtrar por Número de Fases", variable= var20, onvalue='NFASES',offvalue='0')
c20.deselect()
c21=Checkbutton(meu_frame4, text="Filtrar por Formato", variable= var21, onvalue='FORMATO',offvalue='0')
c21.deselect()


#PACKS E PLACES DOS CHECK BUTTONS
c0.grid(row=0,column=0);c1.grid(row=1,column=0);c2.grid(row=2,column=0);c3.grid(row=3,column=0)
c4.grid(row=4,column=0);c5.grid(row=0,column=1);c6.grid(row=1,column=1);c7.grid(row=2,column=1)
c8.grid(row=3,column=1);c9.grid(row=4,column=1);c10.grid(row=0,column=2)
c11.grid(row=1,column=2);c12.grid(row=2,column=2);c13.grid(row=3,column=2)
c14.grid(row=4,column=2);c15.grid(row=0,column=3);c16.grid(row=1,column=3)
c17.grid(row=2,column=3);c18.grid(row=3,column=3);c19.grid(row=0,column=0)
c20.grid(row=0,column=1);c21.grid(row=0,column=2)




#INSERINDO COMBO BOX
vlist = list(df_final.index)
 
Combo = ttk.Combobox(meu_frame1, values = vlist)
Combo.set("Escolha um poço")
Combo.pack(padx=10,pady=6)


#INSERINDO RADIO BUTTON
rb=IntVar()


radio1=Radiobutton(meu_frame3,variable=rb,text="Distancia Euclidiana", value=1,command=radio)
radio2=Radiobutton(meu_frame3,variable=rb,text="Similaridade por Cosseno", value=2,command=radio)
radio1.pack(side='left',padx=10)
radio2.pack(side='left',padx=10)
radio1.select()

#QUADRO DE TEXTO
#tex = Text(master=canv,width=40,height=20)
#tex.grid()


#INSERINDO BOTÕES
btn2 = ttk.Button(meu_frame2, text="Selecionar variaveis",command=filtro,width=20)
btn2.grid(row=6,column=0,)


btn = ttk.Button(meu_frame1, text="Selecionar poço",command=pega,width=15)
btn.pack(pady=5)

btn3 = ttk.Button(meu_frame4, text="Selecionar filtros",command=filtro2,width=15)
btn3.grid(row=1,column=0)


btn4 = ttk.Button(root, text="Sair",command=root.destroy,width=15)
btn4.place(relx="0.095",rely="0.85")

btn5 = ttk.Button(root, text="Ver poço",command=texto,width=15)
btn5.place(relx="0",rely="0.8")

btn6 = ttk.Button(root, text='Limpar Texto', command=lambda: tex.delete(1.0,END),width=15)
btn6.place(relx="0.095",rely="0.8")

btn7 = ttk.Button(root, text='Resultado', command=resultado,width=15)
btn7.place(relx="0.19",rely="0.8")

btn8 = ttk.Button(root, text='Reiniciar', command=leitura,width=15)
btn8.place(relx="0",rely="0.85")
     

root.mainloop()